In [1]:
%load_ext autoreload
%autoreload 2

# Mobile Money Agent Features

All features derived from `mobilemoney_agents_for_upload_win.csv` are calculated here, and then added to the `Transformer` in `src/features/mobile_money.py`

### Ideas (TODO)

* ...

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

In [3]:
train_df = pd.read_csv('../../data/raw/training.csv', index_col=0)
train_df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,...,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification
ID,,,,,,,,,,,,,,,,,,,,,
5086,98,2,3,1,1,2,2,0,0,0,...,-1,4,4,-4.460442,29.811396,0,0,0,0,0
1258,40,1,1,3,5,1,1,1,0,0,...,4,1,4,-6.176438,39.244871,1,1,1,0,3
331,18,2,4,6,3,2,1,0,0,0,...,-1,1,1,-6.825702,37.652798,1,0,0,0,2
6729,50,1,1,3,1,1,1,0,0,0,...,-1,1,4,-3.372049,35.808307,1,0,1,0,3
8671,34,1,1,1,1,2,1,0,1,0,...,-1,1,4,-7.179645,31.039095,1,1,0,1,3


In [4]:
mm_df = pd.read_csv('../../data/raw/FSDT_FinAccessMapping/mobilemoney_agents_for_upload_win.csv')
print(mm_df.shape)
mm_df.head()

(45429, 112)


,region,district,ward,latitude,longitude,agent_name,till_operator,agent_type,standalone_or_other_business,other_business_type,...,other_local_services,other_local_services_details,branding_but_no_service,branding_but_no_service_mpesa,branding_but_no_service_tigo_pesa,branding_but_no_service_ezy_pesa,branding_but_no_service_airtel_money,branding_but_no_service_max_malipo,branding_but_no_service_selcom,branding_but_no_service_button Pay
0,Ruvuma,Tunduru,Kalulu,-10.28298,40.18363,Yusuph Shop,employee,Aggregated,Standalone agent,NaN,...,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
1,Ruvuma,Tunduru,Kalulu,-10.28457,40.18295,Abdulazak Shop,owner,Aggregated,Other business,Duka,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kilimanjaro,Moshi Urban,Bondeni,-3.36025,37.33803,Hussein Ramadhani Shop,employee,Independent,Standalone agent,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dar es Salaam,Kinondoni,Manzese,-6.80213,39.22978,Mechmaster Ltd,employee,Aggregated,Standalone agent,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kilimanjaro,Moshi Urban,Kiusa,-3.35613,37.33382,Bernadeta Mtui,owner,Independent,Other business,Duka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
list(mm_df)

['region',
 'district',
 'ward',
 'latitude',
 'longitude',
 'agent_name',
 'till_operator',
 'agent_type',
 'standalone_or_other_business',
 'other_business_type',
 'trading_hours',
 'provider_tigo_pesa',
 'deposits_tigo_pesa',
 'withdrawls_tigo_pesa',
 'agent_numbers_tigo_pesa',
 'tigo_pesa_agent_number1',
 'tigo_pesa_agent_number2',
 'tigo_pesa_agent_number3',
 'tigo_pesa_balance_your_float',
 'tigo_pesa_training',
 'tigo_pesa_training_type',
 'tigo_pesa_training_tariff_guide',
 'tigo_pesa_log_books',
 'tigo_pesa_visible_guidance',
 'tigo_pesa_visible_guidance_complaints',
 'tigo_pesa_most_common_complaint',
 'tigo_pesa_most_common_complaint_detail',
 'provider_airtel_cash',
 'deposits_airtel_cash',
 'withdrawls_airtel_cash',
 'agent_numbers_airtel_cash',
 'airtel_cash_agent_number1',
 'airtel_cash_agent_number2',
 'airtel_cash_agent_number3',
 'airtel_cash_balance_your_float',
 'airtel_cash_training',
 'airtel_cash_training_type',
 'airtel_cash_training_tariff_guide',
 'airtel_cash

## Create Search Tree

In [6]:
from pysal.lib.cg import KDTree, RADIUS_EARTH_KM

mm_tree = KDTree(mm_df[['latitude','longitude']].values, distance_metric='ARC', radius=RADIUS_EARTH_KM)

In [7]:
useful_cols_base = [
    'region',
    'district',
    'ward',
    'latitude',
    'longitude',
    'agent_type',
    'standalone_or_other_business',
    'other_business_type',
    'trading_hours'
]


other_cols = [
    'provider_tigo_pesa',
    'deposits_tigo_pesa',
    'withdrawls_tigo_pesa',
    'tigo_pesa_training',
    'tigo_pesa_visible_guidance',
    'provider_airtel_cash',
    'deposits_airtel_cash',
    'withdrawls_airtel_cash',
    'airtel_cash_training',
    'airtel_cash_log_books',
    'provider_ezy_pesa',
    'deposits_ezy_pesa',
    'withdrawls_ezy_pesa',
]

# pick more as this progresses
[c for c in list(mm_df) if 'deposit' in c]

['deposits_tigo_pesa',
 'deposits_airtel_cash',
 'deposits_ezy_pesa',
 'deposits_m_pesa']

## Confirm Distance Matching

Find the closest mobile money agent using the KDTree and confirm that its working by showing the customer and matched agent on a map.

In [8]:
import folium

# Select a random coordinate
test_coords = train_df.iloc[np.random.randint(0, len(train_df))][['Latitude','Longitude']]

dist, index = mm_tree.query(test_coords)

# Get coords of closest atm
mm_coords = tuple(mm_df.iloc[index][['latitude','longitude']])


# Create map
m = folium.Map(location=test_coords, height="80%")

# Train marker
folium.CircleMarker(test_coords, radius=10, popup='train',fill=True, fill_color='red', line_color='red').add_to(m)

# ATM marker
folium.CircleMarker(mm_coords, radius=10, popup='ATM').add_to(m)


m

## Create Features

In [9]:
from tqdm import tqdm
tqdm.pandas()

train_coords = train_df[['Latitude','Longitude']]

### Closest Agent Features

In [10]:
withtrawals_cols = ['withdrawls_tigo_pesa',
                         'withdrawls_airtel_cash',
                         'withdrawls_ezy_pesa',
                         'withdrawls_m_pesa']

deposit_cols = ['deposits_tigo_pesa',
                     'deposits_airtel_cash',
                     'deposits_ezy_pesa',
                     'deposits_m_pesa']


def closest_agent_features(tree, coords):
    dist, indx = tree.query(coords)
    
    match = mm_df.iloc[indx]
    
    total_withdraw = sum(match[withtrawals_cols].fillna(0))
    
    total_deposit = sum(match[deposit_cols].fillna(0))
               
    ratio = total_withdraw / max(1,total_deposit)
    
    features = {
        'mm_dist_closest': dist,
        'mm_tigo': match['provider_tigo_pesa'],
        'mm_airtel': match['provider_airtel_cash'],
        'mm_m_pesa': match['provider_m_pesa'],
        'mm_ezy_pesa': match['provider_ezy_pesa'],
        'mm_loans': match['top2_utility_accounts_Loans'],
        'mm_insurance': match['top2_utility_accounts_insurance'],
        'mm_total_withdrawals': total_withdraw,
        'mm_total_deposits': total_deposit,
        'mm_wdraw_dep_ratio': ratio
    }
    
    return features

# Test
test_coords = train_coords.iloc[np.random.randint(0, len(train_df))]
closest_agent_features(mm_tree, test_coords)

{'mm_dist_closest': 15.233987855104662,
 'mm_tigo': nan,
 'mm_airtel': nan,
 'mm_m_pesa': 'Yes',
 'mm_ezy_pesa': nan,
 'mm_loans': nan,
 'mm_insurance': nan,
 'mm_total_withdrawals': 6.0,
 'mm_total_deposits': 5.0,
 'mm_wdraw_dep_ratio': 1.2}

### Number of agents in radius

In [11]:
def num_agents_in_radius(tree, coords, radius=5):
    
    # Get the closest atms
    closest = tree.query_ball_point(coords,r=radius)
    
    # Number of closest
    n_closest = len(closest)
#     n_insurance = mm_df.iloc[closest].fillna(0).replace('Yes', 1)['top2_utility_accounts_insurance'].sum()
    
    features = {
        'mm_n_closest_{}_km'.format(radius): n_closest,
#         f'n_insurance_{radius}_km': n_insurance # ADD LATER
    }
    
    return features

test_coords = train_coords.iloc[np.random.randint(0, len(train_df))]
num_agents_in_radius(mm_tree, test_coords, radius=10)

{'mm_n_closest_10_km': 0}

## Combine Features using one Function

In [12]:
def mm_features(tree, coords, **kwargs):
    
    all_features = {}
    all_features.update(closest_agent_features(mm_tree, coords))
    
    all_features.update(num_agents_in_radius(mm_tree, coords, **kwargs))
    
    return pd.Series(all_features)#.replace({'Yes': 1, np.nan: -1})

## Extract Features and Save to Disk

For both training and testin datasets, extract features and save under `data/modelling/{train/test}_atm_features.csv` 

### Train Features

In [13]:
train_features = train_coords.progress_apply(lambda coords: mm_features(mm_tree, coords, radius=10), axis=1)
train_features.head()

100%|██████████| 7094/7094 [00:37<00:00, 189.05it/s]


,mm_dist_closest,mm_tigo,mm_airtel,mm_m_pesa,mm_ezy_pesa,mm_loans,mm_insurance,mm_total_withdrawals,mm_total_deposits,mm_wdraw_dep_ratio,mm_n_closest_10_km
ID,,,,,,,,,,,
5086,3.137028,NaN,NaN,Yes,NaN,NaN,NaN,4.0,6.0,0.666667,12.0
1258,1.025637,Yes,NaN,NaN,NaN,NaN,NaN,20.0,40.0,0.500000,278.0
331,0.123305,Yes,Yes,Yes,NaN,NaN,NaN,30.0,36.0,0.833333,781.0
6729,0.025663,NaN,Yes,Yes,NaN,NaN,NaN,59.0,67.0,0.880597,61.0
8671,6.587306,NaN,NaN,Yes,NaN,NaN,NaN,35.0,23.0,1.521739,2.0


In [14]:
train_features.to_csv('../../data/modelling/train/mm_features.csv')

### Test Features

In [15]:
test_df = pd.read_csv('../../data/raw/test.csv', index_col=0)

test_coords = test_df[['Latitude','Longitude']]

test_features = test_coords.progress_apply(lambda coords: mm_features(mm_tree, coords, radius=10), axis=1)

test_features.head()

100%|██████████| 2365/2365 [00:12<00:00, 194.19it/s]


,mm_dist_closest,mm_tigo,mm_airtel,mm_m_pesa,mm_ezy_pesa,mm_loans,mm_insurance,mm_total_withdrawals,mm_total_deposits,mm_wdraw_dep_ratio,mm_n_closest_10_km
ID,,,,,,,,,,,
2352,0.008330,NaN,Yes,Yes,NaN,NaN,NaN,25.0,23.0,1.086957,2149
8208,10.837420,NaN,NaN,Yes,NaN,NaN,NaN,12.0,9.0,1.333333,0
2785,1.854432,NaN,NaN,Yes,NaN,NaN,NaN,18.0,13.0,1.384615,52
2967,7.552144,NaN,NaN,Yes,NaN,NaN,NaN,19.0,9.0,2.111111,5
1697,0.047384,NaN,NaN,Yes,NaN,NaN,NaN,5.0,10.0,0.500000,664


In [16]:
test_features.to_csv('../../data/modelling/test/mm_features.csv')

## Deprecated

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class MMAgentsInVicinity(BaseEstimator, TransformerMixin):

    def __init__(self, mm_coords, radius=5):
        self.tree = self.create_tree(mm_coords)
        self.radius = radius

    def create_tree(self, coords):
        return KDTree(coords, distance_metric='ARC', radius=RADIUS_EARTH_KM)

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None):
        x_coords = X[['Latitude', 'Longitude']]

        agents_in_radius = x_coords.apply(
            lambda coords: len(self.tree.query_ball_point(coords, r=self.radius)), axis=1)

        return pd.DataFrame(agents_in_radius)
    
    
class ColumnExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns=None):
        self.columns = columns
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self,X, y=None, **transform_params):
        assert self.columns is not None, 'ColumnExtractor initialized without list of columns'
        return X[self.columns]
        

## Test the transformers

In [18]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

def extract_targets(train_df):
    X = train_df.drop(['mobile_money', 'savings', 'borrowing', 'insurance', 'mobile_money_classification'], axis=1)
    y = train_df['mobile_money_classification']
    return X, y

X, y = extract_targets(train_df)
X_train, X_test, y_train, y_test = train_test_split(X, y)

CONTINUOUS_COLUMNS = ['Q1']

mm_coords = mm_df[['latitude','longitude']].values

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnExtractor(CONTINUOUS_COLUMNS)),
            ('scale', StandardScaler())
        ])),
        ('agents_in_vicinity', MMAgentsInVicinity(mm_coords, radius=5))
    ])), 
    ('clf',LogisticRegression())
])
pipeline.fit(X_train, y_train)


print(log_loss(y_test, pipeline.predict_proba(X_test)))

1.252190502183589
